# Radio Map

In [1]:
import pickle 
import json
import pandas as pd
import numpy as np
import collections

In [2]:
with open('test.pkl' , 'rb') as fp:
    d = pickle.load(fp)

In [3]:
with open('meta.json' , 'r') as fp:
    meta = json.load(fp)

In [4]:
'''
d has the following structure -> {location : RSSI value}


location - > room number.location in that room.timestamp.random
RSSI value -> a vector of length len(AP_LIST)
'''

'\nd has the following structure -> {location : RSSI value}\n\n\nlocation - > room number.location in that room.timestamp.random\nRSSI value -> a vector of length len(AP_LIST)\n'

In [5]:
AP_LIST = meta['ap_list']
AP_MAP = {AP_LIST[i] : i for i in range(len(AP_LIST))}
NUM_RECORDS = len(d)
DIRECTIONS =  ['north', 'south', 'east', 'west']

In [6]:
df = pd.DataFrame(AP_LIST, columns = ['AP'])
temp = pd.DataFrame(d.values()).transpose()
temp.columns = d.keys()
df = pd.concat([df, temp] , axis = 1)
df = df.set_index('AP')

In [7]:
df.head()

,B7.1.north.1674507393.kxy97,B7.1.north.1674507395.0tmtt,B7.1.north.1674507397.r0ura,B7.1.north.1674507398.gi90k,B7.1.north.1674507400.wprjx,B7.1.east.1674507403.2p762,B7.1.east.1674507405.zzbhg,B7.1.east.1674507407.tmii9,B7.1.east.1674507408.xrgrx,B7.1.east.1674507410.tgcr1,...,B5.4.east.1674507309.5s1ju,B5.4.south.1674507312.5adzv,B5.4.south.1674507314.gawxh,B5.4.south.1674507315.ihv6d,B5.4.south.1674507317.ovfdo,B5.4.south.1674507319.64rdx,B5.4.west.1674507323.qxlrg,B5.4.west.1674507326.fbmcx,B5.4.west.1674507328.rcntc,B5.4.west.1674507329.d4wbl
AP,,,,,,,,,,,,,,,,,,,,,
00:1f:f3:c2:67:9f,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
00:54:af:63:a0:c7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
00:bb:1c:3e:5a:d4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
00:bb:1c:fe:5a:d9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
02:44:76:73:b1:d2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [8]:
# dd = pd.DataFrame(AP_LIST, columns = ['AP'])
# for k in list(d.keys()):
#     temp = pd.DataFrame(d[k], columns=[k])
#     dd = pd.concat([dd, temp] , axis = 1)
# dd = dd.set_index('AP')

In [9]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [10]:
RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
INV_RP_MAP = {i : key for i, key in enumerate(RP_LIST)}
TIMESTEPS = 5

In [11]:
radio_map = {}

for direction in DIRECTIONS:
    arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
    for key in RP_MAP:
        cols = [x for x in df.columns if key + '.' + direction in x]
        if len(cols) == 0:
            temp = np.full((len(AP_LIST) , TIMESTEPS) , fill_value=np.nan, dtype=np.float16)
        elif 0 < len(cols) < TIMESTEPS:
            # print(df[cols].to_numpy().shape, np.full((len(AP_LIST) , TIMESTEPS - len(cols)) , fill_value=np.nan, dtype=np.float16).shape)
            temp = np.concatenate((df[cols].to_numpy() , np.full((len(AP_LIST) , TIMESTEPS - len(cols)) , fill_value=np.nan, dtype=np.float16)), axis = 1)
        else:
            temp = df[cols].to_numpy()
        arr[:,RP_MAP[key],:] = temp 

    radio_map[direction] = np.nan_to_num(arr , nan=-100.0)


In [12]:
radio_map['north']

array([[[-100.,  -84.,  -84.,  -86.,  -86.],
        [ -83.,  -83.,  -88., -100., -100.],
        [ -91.,  -91.,  -91.,  -91.,  -91.],
        ...,
        [-100., -100., -100.,  -94., -100.],
        [-100., -100., -100., -100., -100.],
        [ -90.,  -90.,  -90.,  -90.,  -90.]],

       [[-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        ...,
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.]],

       [[-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        ...,
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.]],

       ...,

       [[-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.],
        [-100., -100., -100., -100., -100.

# Reliability and Similarity

In [13]:
GAMMA = -80  #Threshold above whcih an AP readig is too small
M = 2 #How many of the 5 time readings does the AP value need to be above GAMMA
LAMBDA = 1 / 1000 #Specifies a small amount to be added to hamming distance denominator to avoid zero division error

In [14]:
I = {} #Indicator matrices to check for "reliable" APs

In [15]:
for direction in DIRECTIONS:
    T = (radio_map[direction] > GAMMA).astype(int).sum(axis = 2)
    _I = (T > M).astype(int)
    I[direction] = _I

In [16]:
S = {} #Similarity matrices which stores a score for how "close" two RPs are

In [17]:
def hamming(a, b, LAMBDA = LAMBDA):
    return 1 / (np.count_nonzero(a!=b) + LAMBDA)

In [18]:
for direction in DIRECTIONS:
    _S = np.ndarray((len(RP_MAP) , len(RP_MAP)))
    _I = I[direction]
    for i in range(len(RP_MAP)):
        for j in range(len(RP_MAP)):
            _S[i][j] = hamming(_I[:, i] , _I[:, j])
    S[direction] = _S

# Stability

In [19]:
delta = {} # Stability matrix for assigning new cluster centroid

In [20]:
psi = {}
for direction in DIRECTIONS:
    _psi = np.sum(radio_map[direction], axis = 2) / TIMESTEPS
    psi[direction] = _psi
    # print(_psi.shape)
    _delta = np.sum((radio_map[direction] - _psi[:, :, np.newaxis]) ** 2 , axis = 2) / (TIMESTEPS-1)
    _delta = np.sum(_delta * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0) + LAMBDA))
    # print(_delta)
    delta[direction] = _delta



# Clustering

In [21]:
ETA = 0.02 #Threshold for similarity for a node to belong to a cluster

In [22]:
from pprint import pprint

In [23]:
CH = collections.defaultdict(lambda : collections.defaultdict(set))
FL = collections.defaultdict(lambda : collections.defaultdict(set))

for direction in DIRECTIONS:
    # print(direction , '##################################')
    B = set([i for i in range(len(RP_MAP))])
    Bprime = set([i for i in range(len(RP_MAP))])
    edgeset = set()
    visited = set()
    _S = S[direction]
    k = -1
    while B:
        k += 1
        fl = set()
        node = B.pop()
        
        # print('Candidate : ' , INV_RP_MAP[node])
        for j in Bprime:
            if j != node and _S[j,node] >= ETA:
                fl.add(j)
                if j not in visited:
                    visited.add(j)
                else:
                    edgeset.add(j)
        # pprint([INV_RP_MAP[x] for x in fl])
        B = B - fl
        CH[direction][k] = set([node])
        FL[direction][k] = fl

    K = k
    for k in range(K+1):
        temp = (CH[direction][k] | FL[direction][k])
        cluster = list(temp - edgeset)
        stabilities = delta[direction][cluster]
        CH[direction][k] = set([cluster[np.argmin(stabilities)]])
        FL[direction][k] = temp - CH[direction][k]



In [24]:
clusters = {}
for temp_dir in DIRECTIONS:
    temp = {}
    for key in range(len(CH[temp_dir])):
        temp[INV_RP_MAP[list(CH[temp_dir][key])[0]]] = set([INV_RP_MAP[x] for x in FL[temp_dir][key]])
    clusters[temp_dir] = temp

In [26]:
# clusters = {}
# for temp_dir in DIRECTIONS:
#     temp = {}
#     for key in range(len(CH[temp_dir])):
#         temp[list(CH[temp_dir][key])[0]] = FL[temp_dir][key] 
#     clusters[temp_dir] = temp

In [27]:
# GAMMA = -80  #Threshold above whcih an AP readig is too small
# M = 2 #How many of the 5 time readings does the AP value need to be above GAMMA
# LAMBDA = 1 / 1000 #Specifies a small amount to be added to hamming distance denominator to avoid zero division error
# ETA = 0.02
# DIRECTIONS =  ['north', 'south', 'east', 'west']
# import pandas as pd
# # from rssicore.Utils import find_nth, hamming
# import collections
# AP_LIST = meta['ap_list']
# AP_MAP = {AP_LIST[i] : i for i in range(len(AP_LIST))}
# NUM_RECORDS = len(d)

# df = pd.DataFrame(AP_LIST, columns = ['AP'])
# temp = pd.DataFrame(d.values()).transpose()
# temp.columns = d.keys()
# df = pd.concat([df, temp] , axis = 1)
# df = df.set_index('AP')
# RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
# RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
# INV_RP_MAP = {i : key for i, key in enumerate(RP_LIST)}
# TIMESTEPS = 5
# radio_map = {}
# for direction in DIRECTIONS:
#     arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
#     for key in RP_MAP:
#         print(key + '.' + direction)
#         cols = [x for x in df.columns if key + '.' + direction in x]
#         if len(cols) == 0:
#             temp = np.full((len(AP_LIST) , TIMESTEPS) , fill_value=np.nan, dtype=np.float16)
#         elif 0 < len(cols) < TIMESTEPS:
#             print(df[cols].to_numpy().shape, np.full((len(AP_LIST) , TIMESTEPS - len(cols)) , fill_value=np.nan, dtype=np.float16).shape)
#             temp = np.concatenate((df[cols].to_numpy() , np.full((len(AP_LIST) , TIMESTEPS - len(cols)) , fill_value=np.nan, dtype=np.float16)), axis = 1)
#         else:
#             temp = df[cols].to_numpy()
#         arr[:,RP_MAP[key],:] = temp 
#     radio_map[direction] = arr

# I = {}
# for direction in DIRECTIONS:
#     T = (radio_map[direction] > GAMMA).astype(int).sum(axis = 2)
#     _I = (T > M).astype(int)
#     I[direction] = _I
# S = {}
# for direction in DIRECTIONS:
#     _S = np.ndarray((len(RP_MAP) , len(RP_MAP)))
#     _I = I[direction]
#     for i in range(len(RP_MAP)):
#         for j in range(len(RP_MAP)):
#             _S[i][j] = hamming(_I[:, i] , _I[:, j], LAMBDA)
#     S[direction] = _S
# delta = {}
# psi = {}
# for direction in DIRECTIONS:
#     _psi = np.sum(radio_map[direction], axis = 2) / TIMESTEPS
#     psi[direction] = _psi
#     # print(_psi.shape)
#     _delta = np.sum((radio_map[direction] - _psi[:, :, np.newaxis]) ** 2 , axis = 2) / (TIMESTEPS-1)
#     _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))
#     # print(_delta)
#     delta[direction] = _delta

# CH = collections.defaultdict(lambda : collections.defaultdict(set))
# FL = collections.defaultdict(lambda : collections.defaultdict(set))
# for direction in DIRECTIONS:
#     B = set([i for i in range(len(RP_MAP))])
#     Bprime = set([i for i in range(len(RP_MAP))])
#     edgeset = set()
#     visited = set()
#     _S = S[direction]
#     k = -1
#     while B:
#         k += 1
#         fl = set()
#         node = B.pop()
#         for j in Bprime:
#             if j != node and _S[j,node] >= ETA:
#                 fl.add(j)
#                 if j not in visited:
#                     visited.add(j)
#                 else:
#                     edgeset.add(j)
#         B = B - fl
#         CH[direction][k] = set([node])
#         FL[direction][k] = fl
#     K = k
#     for k in range(K+1):
#         temp = (CH[direction][k] | FL[direction][k])
#         cluster = list(temp - edgeset)
#         stabilities = delta[direction][cluster]
#         CH[direction][k] = set([cluster[np.argmin(stabilities)]])
#         FL[direction][k] = temp - CH[direction][k]
# clusters = {}
# for dir in DIRECTIONS:
#     temp = {}
#     for k in CH[direction]:
#         temp[list(CH[direction][k])[0]] = FL[direction][k]
#     clusters[dir] = temp
# edgenodes = collections.defaultdict(lambda : collections.defaultdict(set))
# for dir in DIRECTIONS:
#     cl = clusters[direction]
#     temp = collections.defaultdict(set)
#     for ch , fl in cl.items():
#         for f in fl:
#             temp[f].add(ch)
#             if len(temp[f])>1:
#                 edgenodes[dir][f] = temp[f]
#     # for key in edgenodes[dir]:
#     #     if len(edgenodes[dir][key]) < 2:
#     #         del edgenodes[dir][key]

# # clusters = {}
# # for temp_dir in DIRECTIONS:
# #     temp = {}
# #     for key in range(len(CH[temp_dir])):
# #         temp[INV_RP_MAP[list(CH[temp_dir][key])[0]]] = set([INV_RP_MAP[x] for x in FL[temp_dir][key]])
# #     clusters[temp_dir] = temp
# final = {'clusters' : clusters, 'sparse' : I, 'radio_map' : psi, 'inv_rp_map' : INV_RP_MAP, 'edgenodes' : edgenodes}